# ADA Project
## Modeling

## 3. RNN

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Load and preprocess the data
data = pd.read_csv('../Data/merged_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)
data = data[data['animal-type'] == 'Wolf']

# Define grid search parameters
learning_rates = [1e-5, 1e-4, 1e-3]
nodes = [32, 64, 128, 256] # number of nodes from 32 to 256, in powers of 2 to avoid too complex model
layers = [3, 5, 7] # only 3, 5, 7 to avoid too complex models and overfitting
batch_size = 32
epochs = 10 # due to computational time, I can only train for 10 epochs and 32 batch size (running on a CPU and this code takes already hours to run on my machine)
patience = 3
time_steps = 30 # different lags have been tested from 5 to 40 and 30 seems to be the best tradeoff between accuracy and avoiding overfitting

# Prepare a dictionary to store the results for each wolf
results = {}

# Function to create overlapping windows
def create_overlapping_windows(data, time_steps):
    X, Y = [], [] 
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:i+time_steps].values)
        Y.append(data.iloc[i+time_steps].values)
    return np.array(X), np.array(Y)

# Process each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    print(f"Processing wolf {wolf_id}")
    
    # Create overlapping windows
    X, y = create_overlapping_windows(group[['location-lat', 'location-long']], time_steps)
    
    # Scale the features
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_X.fit_transform(X.reshape(-1, 2)).reshape(X.shape)
    y_scaled = scaler_y.fit_transform(y)
    
    # Split the data into training and testing
    n_train = int(0.8 * len(X_scaled))
    X_train_scaled, y_train_scaled = X_scaled[:n_train], y_scaled[:n_train]
    X_test_scaled, y_test_scaled = X_scaled[n_train:], y_scaled[n_train:]

    # Initialize the best score to a high value
    best_score = np.inf
    best_params = {}

    # Grid search for hyperparameters
    for lr in learning_rates:
        for node in nodes:
            for layer in layers:
                # Define the model
                model = Sequential()
                
                model.add(SimpleRNN(node, return_sequences=True, input_shape=(time_steps, 2), activation='relu')) # First RNN layer
                
                for _ in range(layer - 2): # -2 because first and last layers are already added
                    model.add(SimpleRNN(node, return_sequences=True, activation='relu'))
                model.add(SimpleRNN(node, activation='relu'))  # Last RNN layer
                
                model.add(Dense(2, activation='linear'))  # Output layer
                
                # Compile the model
                optimizer = Adam(learning_rate=lr)
                model.compile(loss='mse', optimizer=optimizer)
                
                # Fit the model with early stopping
                early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
                history = model.fit(
                    X_train_scaled, y_train_scaled, 
                    validation_split=0.2,  # Using 20% of the training data for validation
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[early_stopping], 
                    verbose=0
                )

                # Evaluate the model on the test data
                score = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
                print(f"Model with LR: {lr}, Nodes: {node}, Layers: {layer} has MSE: {score}")

                # Update best model if the score improved
                if score < best_score:
                    best_score = score
                    best_params = {'lr': lr, 'nodes': node, 'layers': layer}
                    best_model = model

    # Forecast future values using the last available input from training
    current_input = X_train_scaled[-1].reshape(1, time_steps, 2)
    predictions = []
    for _ in range(len(y_test_scaled)):
        next_prediction = best_model.predict(current_input)
        predictions.append(next_prediction[0])
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1, :] = next_prediction[0]

    # Inverse transform predictions to original scale
    predictions_scaled_back = scaler_y.inverse_transform(predictions)
    mse = mean_squared_error(scaler_y.inverse_transform(y_test_scaled), predictions_scaled_back)
    
    results[wolf_id] = {
        'mse': mse,
        'predictions': predictions_scaled_back,
        'actual': scaler_y.inverse_transform(y_test_scaled),
        'best_params': best_params
    }
    
    # Save the best model for each wolf
    best_model.save(f'../Results/RNN results/best_model_for_wolf_{wolf_id}.keras')

# Output the results for each wolf
for wolf_id, res in results.items():
    print(f"Wolf ID: {wolf_id}, Test set MSE: {res['mse']}, Best Params: {res['best_params']}")

/var/folders/cd/f_ps01dx04n4r8lqhy1tjk5r0000gn/T/ipykernel_34599/441536481.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/merged_data.csv')


Processing wolf B042


/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model with LR: 0.0001, Nodes: 64, Layers: 3 has MSE: 0.00522706238552928
Model with LR: 0.0001, Nodes: 64, Layers: 5 has MSE: 0.00645142886787653
Model with LR: 0.0001, Nodes: 64, Layers: 7 has MSE: 0.00939810648560524
Model with LR: 0.0001, Nodes: 128, Layers: 3 has MSE: 0.005186303984373808
Model with LR: 0.0001, Nodes: 128, Layers: 5 has MSE: 0.006468864157795906
Model with LR: 0.0001, Nodes: 128, Layers: 7 has MSE: 0.008459855802357197
Model with LR: 0.0001, Nodes: 256, Layers: 3 has MSE: 0.005864238832145929
Model with LR: 0.0001, Nodes: 256, Layers: 5 has MSE: 0.006135805509984493
Model with LR: 0.0001, Nodes: 256, Layers: 7 has MSE: 0.007640660274773836
Model with LR: 0.001, Nodes: 64, Layers: 3 has MSE: 0.005017484538257122
Model with LR: 0.001, Nodes: 64, Layers: 5 has MSE: 0.004827830009162426
Model with LR: 0.001, Nodes: 64, Layers: 7 has MSE: 0.005564036779105663
Model with LR: 0.001, Nodes: 128, Layers: 3 has MSE: 0.004570198245346546
Model with LR: 0.001, Nodes: 128, Laye

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Model with LR: 0.0001, Nodes: 64, Layers: 3 has MSE: 0.004686500411480665
Model with LR: 0.0001, Nodes: 64, Layers: 5 has MSE: 0.0107084596529603
Model with LR: 0.0001, Nodes: 64, Layers: 7 has MSE: 0.012169251218438148
Model with LR: 0.0001, Nodes: 128, Layers: 3 has MSE: 0.007325049489736557
Model with LR: 0.0001, Nodes: 128, Layers: 5 has MSE: 0.01092236116528511
Model with LR: 0.0001, Nodes: 128, Layers: 7 has MSE: 0.012275130487978458
Model with LR: 0.0001, Nodes: 256, Layers: 3 has MSE: 0.005564046557992697
Model with LR: 0.0001, Nodes: 256, Layers: 5 has MSE: 0.008283332921564579
Model with LR: 0.0001, Nodes: 256, Layers: 7 has MSE: 0.009455096907913685
Model with LR: 0.001, Nodes: 64, Layers: 3 has MSE: 0.0052498276345431805
Model with LR: 0.001, Nodes: 64, Layers: 5 has MSE: 0.004801030736416578
Model with LR: 0.001, Nodes: 64, Layers: 7 has MSE: 0.00993089284747839
Model with LR: 0.001, Nodes: 128, Layers: 3 has MSE: 0.0047044772654771805
Model with LR: 0.001, Nodes: 128, Lay

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model with LR: 0.0001, Nodes: 64, Layers: 3 has MSE: 0.09113117307424545
Model with LR: 0.0001, Nodes: 64, Layers: 5 has MSE: 0.10383085906505585
Model with LR: 0.0001, Nodes: 64, Layers: 7 has MSE: 0.10231465846300125
Model with LR: 0.0001, Nodes: 128, Layers: 3 has MSE: 0.02989000827074051
Model with LR: 0.0001, Nodes: 128, Layers: 5 has MSE: 0.06406307220458984
Model with LR: 0.0001, Nodes: 128, Layers: 7 has MSE: 0.10017509013414383
Model with LR: 0.0001, Nodes: 256, Layers: 3 has MSE: 0.03397860750555992
Model with LR: 0.0001, Nodes: 256, Layers: 5 has MSE: 0.07269619405269623
Model with LR: 0.0001, Nodes: 256, Layers: 7 has MSE: 0.08972319215536118
Model with LR: 0.001, Nodes: 64, Layers: 3 has MSE: 0.07223395258188248
Model with LR: 0.001, Nodes: 64, Layers: 5 has MSE: 0.0764421746134758
Model with LR: 0.001, Nodes: 64, Layers: 7 has MSE: 0.12632912397384644
Model with LR: 0.001, Nodes: 128, Layers: 3 has MSE: 0.06246849521994591
Model with LR: 0.001, Nodes: 128, Layers: 5 has M

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model with LR: 0.0001, Nodes: 64, Layers: 3 has MSE: 0.013793813064694405
Model with LR: 0.0001, Nodes: 64, Layers: 5 has MSE: 0.030204370617866516
Model with LR: 0.0001, Nodes: 64, Layers: 7 has MSE: 0.019920727238059044
Model with LR: 0.0001, Nodes: 128, Layers: 3 has MSE: 0.007387702818959951
Model with LR: 0.0001, Nodes: 128, Layers: 5 has MSE: 0.008170804008841515
Model with LR: 0.0001, Nodes: 128, Layers: 7 has MSE: 0.0173951368778944
Model with LR: 0.0001, Nodes: 256, Layers: 3 has MSE: 0.007176581770181656
Model with LR: 0.0001, Nodes: 256, Layers: 5 has MSE: 0.008172840811312199
Model with LR: 0.0001, Nodes: 256, Layers: 7 has MSE: 0.010253663174808025
Model with LR: 0.001, Nodes: 64, Layers: 3 has MSE: 0.006827762350440025
Model with LR: 0.001, Nodes: 64, Layers: 5 has MSE: 0.0054742200300097466
Model with LR: 0.001, Nodes: 64, Layers: 7 has MSE: 0.011517643928527832
Model with LR: 0.001, Nodes: 128, Layers: 3 has MSE: 0.005934759974479675
Model with LR: 0.001, Nodes: 128, La

#### Map of prediction

In [7]:
import folium
import pandas as pd

# Function to create a map for a single wolf
def create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id):
    # Starting point for the map
    start_lat = wolf_data['location-lat'].iloc[0]
    start_long = wolf_data['location-long'].iloc[0]
    wolf_map = folium.Map(location=[start_lat, start_long], zoom_start=8)

    # Split the data into training and testing
    split_index = int(len(wolf_data) * 0.8)
    train_data = wolf_data.iloc[:split_index]
    test_data = wolf_data.iloc[split_index:]

    # Define the HTML code for a purple cross
    html_purple_cross = '''
    <div style="position: relative; width: 8px; height: 8px;">
        <div style="position: absolute; top: 50%; left: 0; transform: translate(0%, -50%); width: 100%; height: 2px; background-color: purple;"></div>
        <div style="position: absolute; top: 0; left: 50%; transform: translate(-50%, 0%); width: 2px; height: 100%; background-color: purple;"></div>
    </div>
    '''

    # Add training data points
    train_points = [(row['location-lat'], row['location-long']) for idx, row in train_data.iterrows()]
    folium.PolyLine(train_points, color="blue", weight=2.5, opacity=1).add_to(wolf_map)
    for point in train_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue',
            popup='Train'
        ).add_to(wolf_map)

    # Add test data points
    test_points = [(row['location-lat'], row['location-long']) for idx, row in test_data.iterrows()]
    folium.PolyLine(test_points, color="green", weight=2.5, opacity=1).add_to(wolf_map)
    for point in test_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='green',
            fill=True,
            fill_color='green',
            popup='Test'
        ).add_to(wolf_map)

    # Mark the first test observation with a purple cross
    folium.Marker(
        location=test_points[0],
        icon=folium.DivIcon(html=html_purple_cross),
        popup='First Test Observation'
    ).add_to(wolf_map)

    # Add prediction points and draw lines between them
    prediction_points = list(zip(test_preds_lat, test_preds_long))
    folium.PolyLine(prediction_points, color="red", weight=2.5, opacity=1).add_to(wolf_map)
    for idx, point in enumerate(prediction_points):
        folium.CircleMarker(
            location=point,
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            popup=f'Predicted: {idx}'
        ).add_to(wolf_map)

    # Save the map in a specific directory for RNN results
    wolf_map.save(f'../Visualisation/RNN/wolf_{wolf_id}_map.html')

# Iterate through each wolf and create a map
for wolf_id, info in results.items():
    test_preds_lat = info['predictions'][:, 0]  
    test_preds_long = info['predictions'][:, 1]
    wolf_data = data[data['individual-id'] == wolf_id]
    create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id)


#### RNN accuracy for each wolf

In [5]:
import numpy as np
from math import radians, cos, sin, asin, sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers. Adjust if using a different radius
    return c * r

for wolf_id, res in results.items():
    test_data = res['actual']
    predictions = res['predictions']
    
    # Extract latitude and longitude from predictions and actual data
    lat_preds, lon_preds = predictions[:, 0], predictions[:, 1]
    lat_actual, lon_actual = test_data[:, 0], test_data[:, 1]

    # Calculate the Haversine distance for each prediction
    distances = [haversine(lon_p, lat_p, lon_a, lat_a) for lon_p, lat_p, lon_a, lat_a in zip(lon_preds, lat_preds, lon_actual, lat_actual)]
    
    # Calculate metrics
    mae_lat = mean_absolute_error(lat_actual, lat_preds)
    rmse_lat = sqrt(mean_squared_error(lat_actual, lat_preds))
    mae_lon = mean_absolute_error(lon_actual, lon_preds)
    rmse_lon = sqrt(mean_squared_error(lon_actual, lon_preds))
    
    mean_distance = np.mean(distances)
    median_distance = np.median(distances)
    
    # Store extended metrics in the results dictionary
    res.update({
        'MAE_Latitude': mae_lat,
        'RMSE_Latitude': rmse_lat,
        'MAE_Longitude': mae_lon,
        'RMSE_Longitude': rmse_lon,
        'Mean_Haversine_Distance': mean_distance,
        'Median_Haversine_Distance': median_distance
    })

# Print the metrics for each wolf
for wolf_id, info in results.items():
    print(f"RNN - Wolf ID: {wolf_id}")
    print(f"Best Model: Learning Rate: {info['best_params']['lr']}, Nodes: {info['best_params']['nodes']}, Layers: {info['best_params']['layers']}")
    print(f"MAE Latitude: {info['MAE_Latitude']:.4f}")
    print(f"RMSE Latitude: {info['RMSE_Latitude']:.4f}")
    print(f"MAE Longitude: {info['MAE_Longitude']:.4f}")
    print(f"RMSE Longitude: {info['RMSE_Longitude']:.4f}")
    print(f"Mean Haversine Distance: {info['Mean_Haversine_Distance']:.2f} km")
    print(f"Median Haversine Distance: {info['Median_Haversine_Distance']:.2f} km")
    print("----------")


RNN - Wolf ID: B042
Best Model: Learning Rate: 0.001, Nodes: 256, Layers: 3
MAE Latitude: 0.1267
RMSE Latitude: 0.1307
MAE Longitude: 0.0993
RMSE Longitude: 0.1141
Mean Haversine Distance: 16.02 km
Median Haversine Distance: 15.66 km
----------
RNN - Wolf ID: B045
Best Model: Learning Rate: 0.001, Nodes: 256, Layers: 3
MAE Latitude: 0.0349
RMSE Latitude: 0.0544
MAE Longitude: 0.0968
RMSE Longitude: 0.1142
Mean Haversine Distance: 8.89 km
Median Haversine Distance: 8.27 km
----------
RNN - Wolf ID: B065
Best Model: Learning Rate: 0.001, Nodes: 64, Layers: 3
MAE Latitude: 0.2950
RMSE Latitude: 0.3132
MAE Longitude: 0.3342
RMSE Longitude: 0.3482
Mean Haversine Distance: 40.95 km
Median Haversine Distance: 39.79 km
----------
RNN - Wolf ID: B077
Best Model: Learning Rate: 0.001, Nodes: 256, Layers: 3
MAE Latitude: 0.0674
RMSE Latitude: 0.0750
MAE Longitude: 0.1561
RMSE Longitude: 0.2114
Mean Haversine Distance: 14.70 km
Median Haversine Distance: 12.27 km
----------
RNN - Wolf ID: B078
Bes

#### RNN average accuracy

In [6]:
# Initialize variables to accumulate the metrics
total_mae_lat = 0
total_rmse_lat = 0
total_mae_long = 0
total_rmse_long = 0
total_mean_haversine_distance = 0
total_median_haversine_distance = 0
num_wolves = len(results)

# Loop over each wolf's results
for wolf_id, info in results.items():
    # Accumulate individual metrics
    total_mae_lat += info['MAE_Latitude']
    total_rmse_lat += info['RMSE_Latitude']
    total_mae_long += info['MAE_Longitude']
    total_rmse_long += info['RMSE_Longitude']
    total_mean_haversine_distance += info['Mean_Haversine_Distance']
    total_median_haversine_distance += info['Median_Haversine_Distance']

# Calculate average metrics
average_mae_lat = total_mae_lat / num_wolves
average_rmse_lat = total_rmse_lat / num_wolves
average_mae_long = total_mae_long / num_wolves
average_rmse_long = total_rmse_long / num_wolves
average_mean_haversine_distance = total_mean_haversine_distance / num_wolves
average_median_haversine_distance = total_median_haversine_distance / num_wolves

# Print average metrics
print("RNN - Average Metrics for All Wolves:")
print(f"Average MAE Latitude: {average_mae_lat:.4f}")
print(f"Average RMSE Latitude: {average_rmse_lat:.4f}")
print(f"Average MAE Longitude: {average_mae_long:.4f}")
print(f"Average RMSE Longitude: {average_rmse_long:.4f}")
print(f"Average Mean Haversine Distance: {average_mean_haversine_distance:.2f} km")
print(f"Average Median Haversine Distance: {average_median_haversine_distance:.2f} km")

RNN - Average Metrics for All Wolves:
Average MAE Latitude: 0.1132
Average RMSE Latitude: 0.1271
Average MAE Longitude: 0.1996
Average RMSE Longitude: 0.2270
Average Mean Haversine Distance: 20.04 km
Average Median Haversine Distance: 19.36 km


#### Saving the results 

In [8]:
import pandas as pd

for wolf_id, group in data.groupby('individual-id'):

    # Convert results dictionary to a DataFrame for easier CSV saving
    results_df = pd.DataFrame([results[wolf_id]])

    # Save results to a CSV file
    results_csv_path = f'../Results/RNN results/results_{wolf_id}.csv'
    results_df.to_csv(results_csv_path, index=False)